In [1]:
from keras.applications.inception_v3 import InceptionV3
#from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K
from keras import metrics
from keras.optimizers import Adam
import numpy as np
from os import walk
import os
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
from scipy import misc
import sys
import numpy as np
import os
import pickle
from libtiff import TIFF
import libtiff
libtiff.libtiff_ctypes.suppress_warnings()
import glob
import pandas as pd
import random
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
random.seed(2001)
print('Importing Done')

Using TensorFlow backend.


Importing Done


In [2]:
#Extracting Labels
df=pd.read_csv('/home/ictd/Desktop/Arpit/Satellite project/scratch/LANDSAT/Labels/VillageLabels_BF.csv')
village_code=df["Town/Village"].values
emp_label=df["Village_HHD_Cluster_BF"].values
actual_labels= [ int(c) for c in emp_label]
s1 = pd.Series(actual_labels,index=list(village_code))

resizeDim=224
nchannels=3
batch_size=64
numclasses=3

In [3]:
#------------------------------------Pre-Processing--------------------------------------------
#Importing the images

# Test files
image_path_testing = '/home/ictd/Desktop/Arpit/Satellite project/data/All_Images'
dirs2 = os.listdir(image_path_testing)
test_files = []
for files in dirs2:
    file_path = os.path.join(image_path_testing, files)
    test_files.append(file_path)
global n_test
n_test = len(test_files)
print('Test set length:', n_test)

('Test set length:', 180077)


In [4]:
j=0
ind=0

# Creating function for getting the test data in batches
test_vector=np.arange(n_test)
test_vector=np.asarray(test_vector,dtype=np.int32)
random.shuffle(test_vector)

def get_eval_data():
	global j
	global ind
	global test_vector,test_files,s1

	X=np.array([]).reshape((0,resizeDim,resizeDim, nchannels))
	Y=np.zeros((batch_size,numclasses))
	village_label=np.zeros((batch_size,numclasses))

	while ind< len(test_vector):

		ind=(ind+1)%len(test_vector)
		tif = TIFF.open(test_files[test_vector[ind]], mode='r')
		image = tif.read_image()
		dataAll = np.array(image)
		if(dataAll.shape[0]>resizeDim or dataAll.shape[1]>resizeDim):
			continue

		village_code=int( ((test_files[test_vector[ind]].split('@')[3]).split('.')[0]).split()[0] )
		#print(village_code)

		
		val=0
		try:
			try:
				val=int(s1.loc[village_code])-1
			except:
				continue
		except:
			continue
		data=np.delete(dataAll,[11,12],axis=2)

		band2=data[:,:,0]
		band3=data[:,:,1]
		band4=data[:,:,2]
		combinedData=np.dstack((band2,band3,band4))

		left=(resizeDim-combinedData.shape[0])//2
		right=resizeDim-combinedData.shape[0]-left
		up=(resizeDim-combinedData.shape[1])//2
		down=resizeDim-combinedData.shape[1]-up

		data1=np.lib.pad(combinedData,[(left,right),(up,down),(0,0)],'constant')
		data1=np.reshape(data1,(1,resizeDim, resizeDim,nchannels))
		if np.isnan(data1).any():
			continue
		else:
			X=np.vstack((X,data1))
			Y[j%batch_size,val]=1
			village_label[j%batch_size,val]=village_code

		j+=1
		if j%(64)==0:
			X=np.asarray(X,dtype=np.float32)
			Y=np.asarray(Y,dtype=np.int32)
			village_label = np.asarray(village_label,dtype=np.int32)

			dataset = (X, Y)
			return X,Y, village_label

In [5]:
def F1(y_true, y_pred):
    def recall(y_true, y_pred):
        #Recall metric: Only computes a batch-wise average of recall.
        #Computes the recall, a metric for multi-label classification of how many relevant items are selected
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        #Precision metric: Only computes a batch-wise average of precision.
        #Computes the precision, a metric for multi-label classification of how many selected items are relevant.
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
model_weight = ['/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/weights-improvement-00.hdf5', 
		   '/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/weights-improvement-01.hdf5', 
		   '/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/weights-improvement-02.hdf5', 
		   '/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/weights-improvement-03.hdf5', 
		   '/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/weights-improvement-04.hdf5']

In [58]:
counter=0
for m in model_weight:
	base_model = ResNet50(weights=None, include_top=False)
	x = base_model.output
	x = GlobalAveragePooling2D()(x)
	x = Dense(1024, activation='relu')(x)
	x = Dropout(0.5)(x)
	x = Dense(1024, activation='relu')(x)
	x = Dropout(0.5)(x)
	predictions = Dense(3, activation='softmax')(x)

	model = Model(input=base_model.input, output=predictions)

	print ('Loading Weights...')
	model.load_weights(m)

	print('Compiling Model...')
	model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy',metrics=[F1])

	print ('Starting Testing...................')
	# This test is on test_data and on the model that is trained on 5 epochs.
	test_batch_size=64
	total_test_batch= int(n_test/64)-1
	avg=0
	predicted_y=np.array([])
	actual_y=np.array([])
	actual_village = np.array([])

	for i in range(total_test_batch):
		evalX,evalY,eval_village_code=get_eval_data()
	
		loss,f1 = model.evaluate(evalX,evalY, batch_size=64)
	
		y1=model.predict(evalX,batch_size=64)
	
		y1=np.argmax(y1,axis=1)
		y2=np.argmax(evalY,axis=1)
		y3 = np.amax(eval_village_code,axis=1)
		predicted_y=np.hstack((predicted_y,y1))
		actual_y=np.hstack((actual_y,y2))
		actual_village = np.hstack((actual_village,y3))

		print("loss: ",loss)
		print("F1 Score: ",f1)
		avg+=f1
	print("avg: ", avg/total_test_batch)
	arr=np.vstack((actual_y,predicted_y,actual_village)).T
	#path_file = '/home/cse/mtech/mcs172873/LANDSAT/Resnetweights/BF/test_results/Test_Results_BF-0{}.csv'.format(counter)
    path_file = '/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/Test_Results_BF-0{}.csv'.format(counter)
    
    np.savetxt(path_file,arr)
	counter=counter+1

IndentationError: unindent does not match any outer indentation level (<ipython-input-58-6efc443461e3>, line 49)

In [7]:
counter=0
for m in model_weight:
	base_model = ResNet50(weights=None, include_top=False)
	x = base_model.output
	x = GlobalAveragePooling2D()(x)
	x = Dense(1024, activation='relu')(x)
	x = Dropout(0.5)(x)
	x = Dense(1024, activation='relu')(x)
	x = Dropout(0.5)(x)
	predictions = Dense(3, activation='softmax')(x)

	model = Model(input=base_model.input, output=predictions)

	print ('Loading Weights...')
	model.load_weights(m)

	print('Compiling Model...')
	model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy',metrics=[F1])

	print ('Starting Testing...................')
	# This test is on test_data and on the model that is trained on 5 epochs.
	test_batch_size=64
	total_test_batch= int(n_test/64)-1
	avg=0
	predicted_y=np.array([])
	actual_y=np.array([])
	actual_village = np.array([])

	for i in range(total_test_batch):
		evalX,evalY,eval_village_code=get_eval_data()
	
		loss,f1 = model.evaluate(evalX,evalY, batch_size=64)
	
		y1=model.predict(evalX,batch_size=64)
	
		y1=np.argmax(y1,axis=1)
		y2=np.argmax(evalY,axis=1)
		y3 = np.amax(eval_village_code,axis=1)
		predicted_y=np.hstack((predicted_y,y1))
		actual_y=np.hstack((actual_y,y2))
		actual_village = np.hstack((actual_village,y3))

		print("loss: ",loss)
		print("F1 Score: ",f1)
		avg+=f1
	print("avg: ", avg/total_test_batch)
	arr=np.vstack((actual_y,predicted_y,actual_village)).T
	path_file = '/home/ictd/Desktop/Arpit/Satellite project/weights/resnet_imbalance/bf/Test_Results_BF-0{}.csv'.format(counter)
	np.savetxt(path_file,arr)
	counter=counter+1

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loading Weights...
Compiling Model...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Starting Testing...................


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


64/64 [==============================] - 8s
('loss: ', 1.0155556201934814)
('F1 Score: ', 0.24691355228424072)
64/64 [==============================] - 6s
('loss: ', 0.9677479863166809)
('F1 Score: ', 0.3488371670246124)
64/64 [==============================] - 6s
('loss: ', 0.9454588890075684)
('F1 Score: ', 0.29629626870155334)
64/64 [==============================] - 6s
('loss: ', 0.9011682271957397)
('F1 Score: ', 0.41860464215278625)
64/64 [==============================] - 6s
('loss: ', 0.972003698348999)
('F1 Score: ', 0.24390240013599396)
64/64 [==============================] - 7s
('loss: ', 0.9378972053527832)
('F1 Score: ', 0.37209299206733704)
64/64 [==============================] - 6s
('loss: ', 0.9017750024795532)
('F1 Score: ', 0.2597402334213257)
64/64 [==============================] - 6s
('loss: ', 0.9750666618347168)
('F1 Score: ', 0.22499996423721313)
64/64 [==============================] - 6s
('loss: ', 1.0238367319107056)
('F1 Score: ', 0.3373493552207947)
64/64

64/64 [==============================] - 6s
('loss: ', 0.9730725288391113)
('F1 Score: ', 0.2926828861236572)
64/64 [==============================] - 8s
('loss: ', 0.9540911912918091)
('F1 Score: ', 0.34567901492118835)
64/64 [==============================] - 6s
('loss: ', 1.0319565534591675)
('F1 Score: ', 0.2650601863861084)
64/64 [==============================] - 6s
('loss: ', 1.0075180530548096)
('F1 Score: ', 0.24999995529651642)
64/64 [==============================] - 6s
('loss: ', 0.9061287641525269)
('F1 Score: ', 0.3529411256313324)
64/64 [==============================] - 6s
('loss: ', 0.9718877673149109)
('F1 Score: ', 0.37209299206733704)
64/64 [==============================] - 6s
('loss: ', 0.9316774606704712)
('F1 Score: ', 0.3571428060531616)
64/64 [==============================] - 7s
('loss: ', 0.9230425953865051)
('F1 Score: ', 0.26315784454345703)
64/64 [==============================] - 7s
('loss: ', 0.9333014488220215)
('F1 Score: ', 0.27160489559173584)
64/64

64/64 [==============================] - 6s
('loss: ', 0.9747613668441772)
('F1 Score: ', 0.2999999523162842)
64/64 [==============================] - 6s
('loss: ', 0.9527014493942261)
('F1 Score: ', 0.3855421245098114)
64/64 [==============================] - 6s
('loss: ', 0.9963675737380981)
('F1 Score: ', 0.27160489559173584)
64/64 [==============================] - 7s
('loss: ', 1.0107378959655762)
('F1 Score: ', 0.21951214969158173)
64/64 [==============================] - 6s
('loss: ', 1.0151492357254028)
('F1 Score: ', 0.2025316059589386)
64/64 [==============================] - 6s
('loss: ', 1.0328675508499146)
('F1 Score: ', 0.24390240013599396)
64/64 [==============================] - 6s
('loss: ', 0.96315598487854)
('F1 Score: ', 0.37209299206733704)
64/64 [==============================] - 7s
('loss: ', 0.9719391465187073)
('F1 Score: ', 0.24999995529651642)
64/64 [==============================] - 7s
('loss: ', 0.9548919200897217)
('F1 Score: ', 0.32941174507141113)
64/64 

...

64/64 [==============================] - 6s
('loss: ', 0.8246868848800659)
('F1 Score: ', 0.559999942779541)
64/64 [==============================] - 7s
('loss: ', 0.9449344873428345)
('F1 Score: ', 0.49504944682121277)
64/64 [==============================] - 6s
('loss: ', 0.9047720432281494)
('F1 Score: ', 0.5544553995132446)
64/64 [==============================] - 7s
('loss: ', 0.8237463235855103)
('F1 Score: ', 0.5436892509460449)
64/64 [==============================] - 6s
('loss: ', 0.9334890842437744)
('F1 Score: ', 0.4854368567466736)
64/64 [==============================] - 6s
('loss: ', 0.7539263963699341)
('F1 Score: ', 0.5940593481063843)
64/64 [==============================] - 7s
('loss: ', 0.8371701240539551)
('F1 Score: ', 0.5882352590560913)
64/64 [==============================] - 6s
('loss: ', 0.9053454995155334)
('F1 Score: ', 0.42553189396858215)
64/64 [==============================] - 6s
('loss: ', 0.8788553476333618)
('F1 Score: ', 0.4999999403953552)
64/64 [==

64/64 [==============================] - 6s
('loss: ', 0.7885285019874573)
('F1 Score: ', 0.6792452335357666)
64/64 [==============================] - 6s
('loss: ', 0.7740285396575928)
('F1 Score: ', 0.6153845191001892)
64/64 [==============================] - 7s
('loss: ', 0.8808515071868896)
('F1 Score: ', 0.5242717862129211)
64/64 [==============================] - 6s
('loss: ', 0.8284244537353516)
('F1 Score: ', 0.6238531470298767)
64/64 [==============================] - 7s
('loss: ', 0.8051027059555054)
('F1 Score: ', 0.601941704750061)
64/64 [==============================] - 7s
('loss: ', 0.8159146904945374)
('F1 Score: ', 0.5656565427780151)
64/64 [==============================] - 6s
('loss: ', 0.8860951066017151)
('F1 Score: ', 0.5208332538604736)
64/64 [==============================] - 6s
('loss: ', 0.8351902961730957)
('F1 Score: ', 0.5769230127334595)
64/64 [==============================] - 7s
('loss: ', 0.8510864973068237)
('F1 Score: ', 0.611111044883728)
64/64 [=====

64/64 [==============================] - 6s
('loss: ', 0.814078688621521)
('F1 Score: ', 0.5416666269302368)
64/64 [==============================] - 6s
('loss: ', 0.9355056285858154)
('F1 Score: ', 0.5333333015441895)
64/64 [==============================] - 7s
('loss: ', 0.9035559296607971)
('F1 Score: ', 0.49504944682121277)
64/64 [==============================] - 7s
('loss: ', 0.8056634664535522)
('F1 Score: ', 0.5454545021057129)
64/64 [==============================] - 6s
('loss: ', 0.76844722032547)
('F1 Score: ', 0.6336632966995239)
64/64 [==============================] - 7s
('loss: ', 0.8825910687446594)
('F1 Score: ', 0.5346534252166748)
64/64 [==============================] - 7s
('loss: ', 0.763712465763092)
('F1 Score: ', 0.6481480598449707)
64/64 [==============================] - 8s
('loss: ', 0.9052467942237854)
('F1 Score: ', 0.5306121706962585)
64/64 [==============================] - 7s
('loss: ', 0.8357789516448975)
('F1 Score: ', 0.5684210062026978)
64/64 [======

64/64 [==============================] - 7s
('loss: ', 0.9401737451553345)
('F1 Score: ', 0.4854368567466736)
64/64 [==============================] - 8s
('loss: ', 0.8949721455574036)
('F1 Score: ', 0.5925925374031067)
64/64 [==============================] - 7s
('loss: ', 0.8383756875991821)
('F1 Score: ', 0.625)
64/64 [==============================] - 7s
('loss: ', 0.8774598836898804)
('F1 Score: ', 0.5523809194564819)
64/64 [==============================] - 7s
('loss: ', 0.8506309986114502)
('F1 Score: ', 0.5523809194564819)
64/64 [==============================] - 7s
('loss: ', 0.881149172782898)
('F1 Score: ', 0.5048543214797974)
64/64 [==============================] - 7s
('loss: ', 0.7395744323730469)
('F1 Score: ', 0.6346153616905212)
64/64 [==============================] - 8s
('loss: ', 0.871590256690979)
('F1 Score: ', 0.5490195751190186)
64/64 [==============================] - 7s
('loss: ', 0.9057532548904419)
('F1 Score: ', 0.5333333015441895)
64/64 [==================

TypeError: Failed to open file '/home/ictd/Desktop/Arpit/Satellite project/data/All_Images/Karnataka_2001-0000000000-0000000000@1188700.0@KR-4805@607633.tif'